In [439]:
import torch
from torch import nn
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v2
from torchvision.models import MobileNet_V2_Weights

In [440]:
backbone = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)

for param in backbone.features.parameters():
    param.requires_grad = False

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [441]:
data_transform = transforms.Compose([
    #transforms.Resize(size=(128, 128)),
    #transforms.RandomHorizontalFlip(p=0.7),
    #transforms.RandomRotation(10),
    transforms.ToTensor(),
]) 

In [442]:
import torch
import torchvision.transforms as transforms
from torchvision.transforms import InterpolationMode
import random

class RandomGammaAdjustment:
    def __init__(self, gamma_range=(0.8, 1.2)):
        self.gamma_range = gamma_range

    def __call__(self, img):
        gamma = random.uniform(*self.gamma_range)
        return transforms.functional.adjust_gamma(img, gamma, gain=1)

    def __repr__(self):
        return f"{self.__class__.__name__}(gamma_range={self.gamma_range})"

data_aug = transforms.Compose([
    #transforms.Resize(size=(128, 128), interpolation=InterpolationMode.BILINEAR),
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomVerticalFlip(p=0.2),
    #transforms.RandomRotation(degrees=10),
    #transforms.RandomResizedCrop(size=(128, 128), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    #transforms.ColorJitter(brightness=0.2, contrast=0.3),
    RandomGammaAdjustment(gamma_range=(0.6, 1.3)),
    transforms.ToTensor(),
])


In [443]:
import os
from torchvision import datasets, transforms

train_data_path = "/kaggle/input/lung-ds-preproc/Nodule/train"
validation_data_path = "/kaggle/input/lung-ds-preproc/Nodule/val"

train_data = datasets.ImageFolder(root=train_data_path, transform=data_aug)
validation_data = datasets.ImageFolder(root=validation_data_path, transform=data_transform)

In [291]:
train_data

Dataset ImageFolder
    Number of datapoints: 1885
    Root location: /kaggle/input/lung-ds-preproc/Nodule/train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.2)
               RandomVerticalFlip(p=0.2)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
           )

In [444]:
class_names = train_data.classes
class_names

['0', '1', '2', '3', '4']

In [445]:
class_dict = train_data.class_to_idx
class_dict

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}

In [446]:
BATCH_SIZE=32
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=1,
                              shuffle=True)

val_dataloader = DataLoader(dataset=validation_data,
                             batch_size=BATCH_SIZE,
                             num_workers=1,
                             shuffle=False)

train_dataloader, val_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7bc4b595fd30>,
 <torch.utils.data.dataloader.DataLoader at 0x7bc4b595e500>)

In [447]:
backbone.classifier = nn.Sequential (
    nn.Dropout(0.6),
    nn.Linear(1280, 1024),
    nn.Linear(1024, 512),
    nn.SiLU(),
    nn.Linear(512,256),
    nn.SiLU(),
    nn.Linear(256,128),
    nn.SiLU(),
    nn.Linear(128,64),
    nn.SiLU(),
    nn.Dropout(0.3),
    nn.Linear(64,len(class_names))
)

In [448]:
backbone.classifier = nn.Sequential (
    nn.Dropout(0.3),
    nn.Linear(1280, 1024),
    nn.Linear(1024, 512),
    nn.SiLU(),
    nn.Linear(512,256),
    nn.SiLU(),
    nn.Linear(256,128),
    nn.SiLU(),
    nn.Linear(128,64),
    nn.SiLU(),
    nn.Linear(64,len(class_names)),
)

for param in backbone.classifier.parameters():
    param.requires_grad = True

backbone.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=False)
  (1): Linear(in_features=1280, out_features=1024, bias=True)
  (2): Linear(in_features=1024, out_features=512, bias=True)
  (3): SiLU()
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): SiLU()
  (6): Linear(in_features=256, out_features=128, bias=True)
  (7): SiLU()
  (8): Linear(in_features=128, out_features=64, bias=True)
  (9): SiLU()
  (10): Linear(in_features=64, out_features=5, bias=True)
)

In [449]:
backbone.conv1 = nn.Conv2d(3, 
                           64, 
                           kernel_size=(7, 7), 
                           stride=(2, 2), 
                           padding=(3, 3), 
                           bias=False)

In [450]:
from torch.optim import lr_scheduler

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(backbone.parameters(), lr=1e-4)


In [451]:
from torchinfo import summary
summary(backbone, input_size=[1, 3, 96, 96])

Layer (type:depth-idx)                             Output Shape              Param #
MobileNetV2                                        [1, 5]                    9,408
├─Sequential: 1-1                                  [1, 1280, 3, 3]           --
│    └─Conv2dNormActivation: 2-1                   [1, 32, 48, 48]           --
│    │    └─Conv2d: 3-1                            [1, 32, 48, 48]           (864)
│    │    └─BatchNorm2d: 3-2                       [1, 32, 48, 48]           (64)
│    │    └─ReLU6: 3-3                             [1, 32, 48, 48]           --
│    └─InvertedResidual: 2-2                       [1, 16, 48, 48]           --
│    │    └─Sequential: 3-4                        [1, 16, 48, 48]           (896)
│    └─InvertedResidual: 2-3                       [1, 24, 24, 24]           --
│    │    └─Sequential: 3-5                        [1, 24, 24, 24]           (5,136)
│    └─InvertedResidual: 2-4                       [1, 24, 24, 24]           --
│    │    └─Sequent

In [452]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               device=device):
  model.train()

  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    y_pred = model(X) 


    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

  
    optimizer.zero_grad()

    
    loss.backward()

  
    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class==y).sum().item()/len(y_pred)
  
  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader) 
  return train_loss, train_acc 


In [453]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device=device):
  model.eval()

  test_loss, test_acc = 0,  0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader): 
      X, y = X.to(device), y.to(device)

      test_pred_logits = model(X)

     
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

    
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

In [454]:
from tqdm.auto import tqdm

def train(model: torch.nn.Module,
          train_dataloader,
          val_dataloader,
          optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5, 
          device=device):
  
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}
  
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=val_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)
    
    print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)
  
  return results

In [455]:
NUM_EPOCHS = 15

from timeit import default_timer as timer
start_time = timer() 

model_0_results = train(model=backbone,
                        train_dataloader=train_dataloader,
                        val_dataloader=val_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 1.5677 | Train acc: 0.2362 | Test loss: 1.4232 | Test acc: 0.3763
Epoch: 1 | Train loss: 1.3583 | Train acc: 0.3873 | Test loss: 1.2239 | Test acc: 0.5096
Epoch: 2 | Train loss: 1.2954 | Train acc: 0.4490 | Test loss: 1.1606 | Test acc: 0.5138
Epoch: 3 | Train loss: 1.2310 | Train acc: 0.4931 | Test loss: 1.1733 | Test acc: 0.5218
Epoch: 4 | Train loss: 1.2035 | Train acc: 0.4993 | Test loss: 1.1508 | Test acc: 0.5309
Epoch: 5 | Train loss: 1.1904 | Train acc: 0.4976 | Test loss: 1.1058 | Test acc: 0.5242
Epoch: 6 | Train loss: 1.1761 | Train acc: 0.5004 | Test loss: 1.0937 | Test acc: 0.5454
Epoch: 7 | Train loss: 1.1724 | Train acc: 0.5009 | Test loss: 1.0716 | Test acc: 0.5437
Epoch: 8 | Train loss: 1.1703 | Train acc: 0.5004 | Test loss: 1.0587 | Test acc: 0.5729
Epoch: 9 | Train loss: 1.1571 | Train acc: 0.5153 | Test loss: 1.0381 | Test acc: 0.5617
Epoch: 10 | Train loss: 1.1384 | Train acc: 0.5177 | Test loss: 1.0460 | Test acc: 0.5731
Epoch: 11 | Train lo

In [456]:
NUM_EPOCHS = 15

from timeit import default_timer as timer
start_time = timer() 

for param in backbone.features[-10:].parameters():
    param.requires_grad = True

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(backbone.parameters(), lr=1e-4)

model_1_results = train(model=backbone,
                        train_dataloader=train_dataloader,
                        val_dataloader=val_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)

end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 1.1045 | Train acc: 0.5373 | Test loss: 0.8665 | Test acc: 0.6409
Epoch: 1 | Train loss: 0.8778 | Train acc: 0.6335 | Test loss: 0.7679 | Test acc: 0.6746
Epoch: 2 | Train loss: 0.7585 | Train acc: 0.6885 | Test loss: 0.6935 | Test acc: 0.7138
Epoch: 3 | Train loss: 0.6878 | Train acc: 0.7227 | Test loss: 0.6494 | Test acc: 0.7579
Epoch: 4 | Train loss: 0.6278 | Train acc: 0.7529 | Test loss: 0.6662 | Test acc: 0.7333
Epoch: 5 | Train loss: 0.5691 | Train acc: 0.7943 | Test loss: 0.6202 | Test acc: 0.7688
Epoch: 6 | Train loss: 0.4821 | Train acc: 0.8125 | Test loss: 0.5832 | Test acc: 0.7930
Epoch: 7 | Train loss: 0.4512 | Train acc: 0.8331 | Test loss: 0.6647 | Test acc: 0.7576
Epoch: 8 | Train loss: 0.4095 | Train acc: 0.8431 | Test loss: 0.6635 | Test acc: 0.7646
Epoch: 9 | Train loss: 0.3829 | Train acc: 0.8505 | Test loss: 0.6136 | Test acc: 0.7913
Epoch: 10 | Train loss: 0.3154 | Train acc: 0.8836 | Test loss: 0.7280 | Test acc: 0.7621
Epoch: 11 | Train lo

In [ ]:
NUM_EPOCHS = 15

from timeit import default_timer as timer
start_time = timer() 

for param in backbone.features[-15:].parameters():
    param.requires_grad = True

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(backbone.parameters(), lr=1e-6, momentum=0.9)

model_1_results = train(model=backbone,
                        train_dataloader=train_dataloader,
                        val_dataloader=val_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)

end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 0.5521 | Train acc: 0.7756 | Test loss: 0.6775 | Test acc: 0.7263
Epoch: 1 | Train loss: 0.5359 | Train acc: 0.8037 | Test loss: 0.6819 | Test acc: 0.7242
Epoch: 2 | Train loss: 0.5218 | Train acc: 0.7952 | Test loss: 0.6704 | Test acc: 0.7329
Epoch: 3 | Train loss: 0.5432 | Train acc: 0.7939 | Test loss: 0.6765 | Test acc: 0.7326
